In [2]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import simpy
import random
import statistics

In [5]:
wait_times = []


class Theater(object):
    def __init__(self, env, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)

    def purchase_ticket(self, moviegoer):
        yield self.env.timeout(random.randint(1, 3))

    def check_ticket(self, moviegoer):
        yield self.env.timeout(3 / 60)

    def sell_food(self, moviegoer):
        yield self.env.timeout(random.randint(1, 5))


def go_to_movies(env, moviegoer, theater):
    # Moviegoer arrives at the theater
    arrival_time = env.now

    with theater.cashier.request() as request:
        yield request
        yield env.process(theater.purchase_ticket(moviegoer))

    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))

    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))

    # Moviegoer heads into the theater
    wait_times.append(env.now - arrival_time)


def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Theater(env, num_cashiers, num_servers, num_ushers)

    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, theater))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person

        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))


def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)


def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_servers = input("Input # of servers working: ")
    num_ushers = input("Input # of ushers working: ")
    params = [num_cashiers, num_servers, num_ushers]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. Simulation will use default values:",
            "\n1 cashier, 1 server, 1 usher.",
        )
        params = [1, 1, 1]
    return params


def main():
    # Setup
    random.seed(42)
    num_cashiers, num_servers, num_ushers = get_user_input()

    # Run the simulation
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    env.run(until=90)

    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print(
        "Running simulation...",
        f"\nThe average wait time is {mins} minutes and {secs} seconds.",
    )


if __name__ == "__main__":
    main()

Input # of cashiers working: 7
Input # of servers working: 5
Input # of ushers working: 1
Running simulation... 
The average wait time is 17 minutes and 8 seconds.


In [ ]:
def task(env, name, counter, mean_time_to_connect): # Логирует
  """The task arrives, is served, and leaves."""
  print('%7.4f %s: Here I am' % (env.now, name))
  with counter.request() as req:
    patience = 0
    # Try to connect or abort the attempt
    results = yield req | env.timeout(patience)
    if req in results:
      # We got to the counter
      print('%7.4f %s: started' % (env.now, name))
      tib = random.expovariate(1.0 / mean_time_to_connect)
      yield env.timeout(tib)
      print('%7.4f %s: finished' % (env.now, name))
    else:
      # We reneged
      print('%7.4f %s: RENEGED' % (env.now, name))

def source(env, number, intensity, counter, mean_time_to_connect): # Генерирует заявки в систему
  """Source generates tasks randomly"""
  for i in range(number):
    c = task(env, 'Task_%02d' % i, counter, mean_time_to_connect)
    env.process(c)
    t = random.expovariate(intensity)
    yield env.timeout(t)

In [ ]:
# Горячая линия банка
seed = 42
new_tasks = 120 # Число заявок в систему
intensity_of_tasks = 100.10 # Интенсивность поступления заявок
mean_time_to_connect = 30 # Среднее время обслуживания (Интенсивность обслуживания = 1/mean_time_to_connect)
cap = 100 # Кол-во серверов

# Setup and start the simulation
random.seed(seed)
env = simpy.Environment()
# Start processes and run
counter = simpy.Resource(env, capacity=cap)
env.process(source(env, new_tasks, intensity_of_tasks, counter, mean_time_to_connect))

env.run()

In [ ]:
def example(env):
     event = simpy.events.Timeout(env, delay=1, value=42)
     value = yield event
     print('now=%d, value=%d' % (env.now, value))

env = simpy.Environment()
example_gen = example(env)
p = simpy.events.Process(env, example_gen)

env.run()